In [1]:
import os
import math

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from IPython.display import display
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# Load BSDS500 datsaset

dataset_url = "http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz"
data_dir = keras.utils.get_file(origin = dataset_url, fname = "BSR", untar = True)
root_dir = os.path.join(data_dir, "BSDS500/data")

70770688/70763455 [==============================] - 1s 0us/step


In [3]:
# Create generator dataset object

crop_size = 300
upscale_factor = 3
input_size = crop_size // upscale_factor
batch_size = 8

train_ds = image_dataset_from_directory(
    root_dir,
    batch_size = batch_size,
    image_size = (crop_size, crop_size),
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    label_mode = None,
)

valid_ds = image_dataset_from_directory(
    root_dir,
    batch_size = batch_size,
    image_size = (crop_size, crop_size),
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    label_mode = None,
)

Found 500 files belonging to 2 classes.
Using 400 files for training.
Found 500 files belonging to 2 classes.
Using 100 files for validation.
